In [20]:
import pandas as pd
from tqdm.auto import tqdm
import actions
from pathlib import Path
from utils import *
import re
from tqdm.auto import tqdm

In [21]:
# Define the file paths
parquet_file_abcd = Path("../data/ABCD_tripfiles.parquet")
parquet_file_abcd_conv = Path("../data/ABCD_tripfiles_conv.parquet")
parquet_file_mnop = Path("../data/MNOP_tripfiles.parquet")
parquet_file_mnop_conv = Path("../data/MNOP_tripfiles_conv.parquet")
parquet_file_zyxw = Path("../data/ZYXW_tripfiles.parquet")
parquet_file_zyxw_conv = Path("../data/ZYXW_tripfiles_conv.parquet")
print(
    parquet_file_abcd,
    parquet_file_abcd_conv,
    parquet_file_mnop,
    parquet_file_mnop_conv,
    parquet_file_zyxw,
    parquet_file_zyxw_conv,
)

..\data\ABCD_tripfiles.parquet ..\data\ABCD_tripfiles_conv.parquet ..\data\MNOP_tripfiles.parquet ..\data\MNOP_tripfiles_conv.parquet ..\data\ZYXW_tripfiles.parquet ..\data\ZYXW_tripfiles_conv.parquet


In [22]:
df_abcd = pd.read_parquet(parquet_file_abcd)
df_mnop = pd.read_parquet(parquet_file_mnop)
df_zyxw = pd.read_parquet(parquet_file_zyxw)

In [23]:
print(round(df_abcd.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_mnop.memory_usage(deep=True).sum() / 1024**2, 2), "MB")
print(round(df_zyxw.memory_usage(deep=True).sum() / 1024**2, 2), "MB")

1214.94 MB
2219.62 MB
585.54 MB


In [24]:
action_names = set(df_abcd.action_name.unique())
print(len(action_names))
action_names.update(set(df_mnop.action_name.unique()))
print(len(action_names))
action_names.update(set(df_zyxw.action_name.unique()))
print(len(action_names))

49
62
67


In [25]:
action_extractors = {
    "CalculateWeightAndTrimAction": None,
    "CheckinMsgProcessor": None,
    "CreateLoadsheetAction": None,
    "CreateZFWMessageAction": None,
    "SendFuelOrderAction": None,
    "SendLoadsheetAction": None,
    "SetActualBagWeightIndicatorAction": None,
    "StorePaxDataAction": None,
    "StoreRegistrationAndConfigurationAc": None,
    "UpdateFuelDataAction": None,
    "RampFinalAction": actions.RampFinalAction.extract,
    "CreateLoadingInstructionAction": None,
    "CopyPaxDataAction": None,
    "CreateBaggageLoadItemsAction": None,
    "EstimateStorePaxDataAction": None,
    "SendLoadingInstructionAction": None,
    "SetCKIPaxDistributionAction": actions.SetCKIPaxDistributionAction.extract,
    "SpecialPaxWeightAction": None,
    "StoreAircraftDataAction": None,
    "StoreCKIAverageWeightAction": None,
    "StorePaxDataGuiAction": None,
    "TdmCreateLoadingInstructionAction": None,
    "TransferCargoAction": None,
    "TransferCheckinDataAction": None,
    "UpdateCargoMailEstimatesAction": None,
    "UpdateEstimatesAction": actions.UpdateEstimatesAction.extract,
    "UpdateLastEzfwSentAction": None,
    "UpdateLoadTableAction": None,
}

In [26]:
def extract_df(df: pd.DataFrame):
    tqdm.pandas()
    for action_name, extractor in action_extractors.items():

        if extractor is not None:
            print(action_name)

            df[f"data_{action_name}"] = df[df.action_name == action_name][
                "entry_details"
            ].progress_apply(extractor)
    return df


df_abcd_conv = extract_df(df_abcd)
df_abcd_conv.to_parquet(parquet_file_abcd_conv, engine="pyarrow", compression="brotli")

df_mnop_conv = extract_df(df_mnop)
df_mnop_conv.to_parquet(parquet_file_mnop_conv, engine="pyarrow", compression="brotli")

df_zyxw_conv = extract_df(df_zyxw)
df_zyxw_conv.to_parquet(parquet_file_zyxw_conv, engine="pyarrow", compression="brotli")

RampFinalAction


100%|██████████| 3804/3804 [00:00<00:00, 42362.55it/s]


SetCKIPaxDistributionAction


0it [00:00, ?it/s]

UpdateEstimatesAction


...

100%|██████████| 65/65 [00:00<00:00, 29802.12it/s]


SetCKIPaxDistributionAction


100%|██████████| 11598/11598 [00:00<00:00, 45338.12it/s]


UpdateEstimatesAction


0it [00:00, ?it/s]


In [27]:
df = df_abcd


x = df[
    (df.action_name == "UpdateEstimatesAction")
    # & (~df["entry_details"].isnull())
    # & (
    #     ~df["entry_details"].str.contains(
    #         "com.onesystem.lc2.estimateshandling.dto.EstimateWeightsDTO", na=False
    #     )
    # )
    # & (
    #     ~df["entry_details"].str.contains(
    #         "com.systemone.lc2.estimateshandling.dto.EstimateWeightsDTO", na=False
    #     )
    # )
    & (
        ~df["entry_details"].str.contains("Pax Weight =", na=False)
    )  # don't include this
]["entry_details"].unique()

In [28]:
"""TOTAL Pax: 6    Y: 6  Jump: 0    StandBy: NULL  Male: 2  Female: 2  Child: 2  Infant: 0  Total bag: 6  Total bag weight: 72.0 KG  Baggage weight type: HISTORIC\r\n 

Distribution        : CKI_DISTRIBUTION         \r\n 
Section             : 0A                       0C                       \r\n 
Capacity            : Y72                      Y102                     \r\n 
Distribution        : Y0                       Y0"""

'TOTAL Pax: 6    Y: 6  Jump: 0    StandBy: NULL  Male: 2  Female: 2  Child: 2  Infant: 0  Total bag: 6  Total bag weight: 72.0 KG  Baggage weight type: HISTORIC\r\n \n\nDistribution        : CKI_DISTRIBUTION         \r\n \nSection             : 0A                       0C                       \r\n \nCapacity            : Y72                      Y102                     \r\n \nDistribution        : Y0                       Y0'

In [29]:
foo: set = set()

In [30]:
for message in x:
    items = message.split(" ")
    for item in items:
        if not item.isdigit():
            foo.add(item)

In [31]:
foo

{'',
 '2500.0',
 '16280.0',
 '27.0',
 '44976.0',
 '15423.0',
 '17546.0',
 '18838.0',
 '18404.0',
 '15264.0',
 '11550.0',
 '3150.0',
 '11919.0',
 '20.0',
 '14802.0',
 '12230.0',
 '3000.0',
 '81.0',
 '63300.0',
 '8255.0',
 '19980.0',
 '44710.0',
 '50000.0',
 '146.0',
 '14899.0',
 '457.0',
 '141.0',
 '2490.0',
 '45662.0',
 '10270.0',
 '13763.0',
 '13420.0',
 '15478.0',
 '15654.0',
 '58000.0',
 '2130.0',
 '13017.0',
 '18443.0',
 '99.0',
 '12390.0',
 '44852.0',
 '14538.0',
 '11064.0',
 '7110.0',
 '55.0',
 '13538.0',
 '274.0',
 '308.0',
 '1141.0',
 '3020.0',
 '11915.0',
 '13738.0',
 '8431.0',
 '14435.0',
 '1170.0',
 '832.0',
 '14734.0',
 '984.0',
 '7.0',
 '16102.0',
 '15490.0',
 '2570.0',
 '16562.0',
 '168.0',
 '23.0',
 '12643.0',
 '17037.0',
 '18143.0',
 '14495.0',
 '14195.0',
 '13390.0',
 '14569.0',
 '45430.0',
 '12780.0',
 '55800.0',
 '6375.0',
 '14029.0',
 '12870.0',
 '15999.0',
 '12570.0',
 '15158.0',
 '170.0',
 '1304.0',
 '3454.0',
 '19554.0',
 '10550.0',
 '1150.0',
 '114.0',
 '1887.0'

In [32]:

x["data_UpdateEstimatesAction"] = x[x.action_name == "UpdateEstimatesAction"][
    "entry_details"
].progress_apply(extract)

AttributeError: 'numpy.ndarray' object has no attribute 'action_name'

In [ ]:
x = df_abcd.copy()